In [1]:
import pandas as pd 
import tools as tl 
import time
from selenium.webdriver.support.select import Select
import re
import requests

### Extração

Acessando o portal do exchage para extrair a lista de clientes ativos

In [2]:
driver = tl.create_driver_chrome(headless=False)
driver.get("https://console.emailexchangeonline.com.br")

In [3]:
tl.clicking(element='',path='username', btype='name', driver=driver).send_keys('suporte@abreuhost.com.br')
tl.clicking(element='',path='password', btype='name', driver=driver).send_keys('c3&8A*wE')
tl.clicking(element='',path='/html/body/div[3]/div/div[1]/div/form/button', driver=driver).click()

>>>Sucesso >>>
>>>Sucesso >>>
>>>Sucesso >>>


In [4]:
time.sleep(15)
driver.get("https://console.emailexchangeonline.com.br/clients?_tc=1693309403931")

In [5]:
# Encontrar o elemento seletor usando XPath
selector_element = driver.find_element('xpath', '//*[@id="page-content"]/div[2]/div/div/div/pagination/div/div[1]/div/select')

# Criar um objeto Select a partir do elemento
select = Select(selector_element)

# Selecionar a opção "500" pelo valor
select.select_by_value('number:500')

In [6]:
time.sleep(10)
table = driver.find_element('xpath', '/html/body/div[2]/div/div[5]/div[1]/div/div[2]/div/div/div/div/table')
                                        
# Extrair o conteúdo HTML da tabela
html = table.get_attribute('outerHTML')

driver.quit()

# Converter o HTML em um DataFrame do Pandas
df = pd.read_html(html)[0]

In [7]:
df.to_excel(r'C:\Users\no\Python Programs\abreu_network_robot\files\clientes_exchange.xlsx', index=False)

### Tratamento

Realizando a limpeza dos dados, e tratamento, para aplicar uma iteligencia artificial para retorne o ramo que o cliente trabalha

In [8]:
#Removendo a coluna Unnamed: 6
df.drop(['Unnamed: 6'], axis=1, inplace=True)

In [9]:
#Removendo as linhas com usuarios em branco ("Nenhum usuario encontrado")
blank_index = df.loc[df['| Nome'] == 'Nenhum usuário encontrado'].index
df.drop(blank_index, axis=0, inplace=True)

In [10]:
df.tail()

,| #,| Status,| Tipo,| Nome,| Revenda,| Cliente desde
144,1723,Ativo,Cliente,VITI VINICOLA BELLA AURORA LTDA 06.958.801/00...,GILSON PEREIRA DE ABREU ME,08/02/2023
145,1756,Ativo,Cliente,MARA GOMES LOBO FONTAN 43.595.983/0001-69,GILSON PEREIRA DE ABREU ME,29/03/2023
146,1775,Ativo,Cliente,SUPERMERCADO IRMAOS NAGAI LTDA 55.237.473/000...,GILSON PEREIRA DE ABREU ME,12/05/2023
147,1788,Ativo,Cliente,CLAITON JOSE FLORENCIO DE ALMEIDA 405.672.288-94,GILSON PEREIRA DE ABREU ME,20/06/2023
148,1817,Ativo,Cliente,ESTEVAO GEINES BERNA SANTIAGO 11.476.786/0001-34,GILSON PEREIRA DE ABREU ME,26/07/2023


In [11]:
df_training = df[['| Status', '| Nome']]
df_training.head()

,| Status,| Nome
0,Ativo,LORIVAL POMPIANI MAIRINQUE EPP 74.512.179/000...
1,Desativado,JESSICA TERRA NEGRAO MORAES 24.318.252/0001-21
2,Ativo,CLUBE ATLÉTICO SOROCABANA DE MAIRINQUE 46.881...
3,Desativado,WL EMPILHADEIRAS ME 14.805.057/0001-72
4,Ativo,VINHOS QUINTA DO NINO LTDA 48.727.804/0001-13


In [12]:
# Função para extrair nome, CPF e CNPJ
def extract_nome_cpf_cnpj(row):

    match_cnpj = re.search(r'\d{2}\.\d{3}\.\d{3}/\d{4}-\d{2}', row['| Nome'])
    match_cpf = re.search(r'\d{3}\.\d{3}\.\d{3}-\d{2}', row['| Nome'])
    
    nome = re.sub(r'(\d{2}\.\d{3}\.\d{3}/\d{4}-\d{2}|\d{3}\.\d{3}\.\d{3}-\d{2})', '', row['| Nome']).strip()
    cnpj = match_cnpj.group() if match_cnpj else None
    cpf = match_cpf.group() if match_cpf else None
    
    return nome, cpf, cnpj

# Aplicar a função para extrair nome, CPF e CNPJ
df_training['| Nome'], df_training['| CPF'], df_training['| CNPJ'] = zip(*df.apply(extract_nome_cpf_cnpj, axis=1))

C:\Users\no\AppData\Local\Temp\ipykernel_9164\1589391080.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_training['| Nome'], df_training['| CPF'], df_training['| CNPJ'] = zip(*df.apply(extract_nome_cpf_cnpj, axis=1))
C:\Users\no\AppData\Local\Temp\ipykernel_9164\1589391080.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_training['| Nome'], df_training['| CPF'], df_training['| CNPJ'] = zip(*df.apply(extract_nome_cpf_cnpj, axis=1))
C:\Users\no\AppData\Local\Temp\ipykernel_9164\1589391080.py:14

In [13]:
df_training_ativo = df_training.loc[df_training['| Status'] == 'Ativo']
df_training_ativo

,| Status,| Nome,| CPF,| CNPJ
0,Ativo,LORIVAL POMPIANI MAIRINQUE EPP,None,74.512.179/0001-00
2,Ativo,CLUBE ATLÉTICO SOROCABANA DE MAIRINQUE,None,46.881.553/0001-00
4,Ativo,VINHOS QUINTA DO NINO LTDA,None,48.727.804/0001-13
5,Ativo,CONTAINERS FLADAFI LTDA ME,None,07.246.790/0001-39
6,Ativo,SOLDATOPO CONTAINERS LTDA,None,46.029.633/0001-23
...,...,...,...,...
144,Ativo,VITI VINICOLA BELLA AURORA LTDA,None,06.958.801/0001-40
145,Ativo,MARA GOMES LOBO FONTAN,None,43.595.983/0001-69
146,Ativo,SUPERMERCADO IRMAOS NAGAI LTDA,None,55.237.473/0001-07
147,Ativo,CLAITON JOSE FLORENCIO DE ALMEIDA,405.672.288-94,None


In [15]:
def get_main_activity(cnpj):
    
    url = f"https://receitaws.com.br/v1/cnpj/{cnpj}"
    response = requests.get(url)

    if response.status_code == 200:
        result = response.json()
        return result['atividade_principal'][0]['text']
    elif response.status_code == 429:
        time.sleep(60)
        return get_main_activity(cnpj)

df_cnpj = df_training_ativo.loc[~df_training_ativo['| CNPJ'].isnull()]
cnpj_list = df_cnpj['| CNPJ']
cnpj_list = list(map(lambda cod: cod.replace('.', '').replace('/', '').replace('-', ''), cnpj_list))
df_cnpj['Ramo de Atuação'] = list(map(lambda cnpj: get_main_activity(cnpj), cnpj_list))

200 74512179000100
200 46881553000100
200 48727804000113
429 07246790000139
200 07246790000139
200 46029633000123
200 07607960000163
429 09473373000154
200 09473373000154
200 00380723000163
200 64743529000188
429 66117425000193
200 66117425000193
200 02899400000106
200 07307059000176
429 09190003000100
200 09190003000100
200 15398703000197
200 17364966000100
429 03678841000140
200 03678841000140
200 08478340000134
200 45495694000890
429 00894229000117
200 00894229000117
200 12621229000122
200 13986940000143
429 03543269000102
200 03543269000102
200 11014869000101
200 14448295000178
429 74252545000121
200 74252545000121
200 15666693000123
200 06191994000157
429 00971698000192
200 00971698000192
200 06143996000170
200 11328186000129
429 00476080000156
200 00476080000156
404 00344976400015
200 07011030000142
429 29308243000173
200 29308243000173
200 13112903000106
200 29630901000149
429 01202633000145
200 01202633000145
200 00297566000127
200 02538409000191
429 22634557000117
200 22634557

C:\Users\no\AppData\Local\Temp\ipykernel_9164\2421444784.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cnpj['Ramo de Atuação'] = list(map(lambda cnpj: get_main_activity(cnpj), cnpj_list))


In [ ]:
def get_main_activity(cnpj):
    
    url = f"https://receitaws.com.br/v1/cnpj/{cnpj}"
    response = requests.get(url)

    if response.status_code == 200:
        result = response.json()
        return result['atividade_principal'][0]['text']
    elif response.status_code == 429:
        time.sleep(60)
        return get_main_activity(cnpj)

df_cnpj = df_training_ativo.loc[~df_training_ativo['| CNPJ'].isnull()]
cnpj_list = df_cnpj['| CNPJ']
cnpj_list = list(map(lambda cod: cod.replace('.', '').replace('/', '').replace('-', ''), cnpj_list))
df_cnpj['Ramo de Atuação'] = list(map(lambda cnpj: get_main_activity(cnpj), cnpj_list))

In [ ]:
df_cpf = df_training_ativo.loc[~df_training_ativo['| CPF'].isnull()]

In [ ]:
df_all = pd.concat([df_cnpj, df_cpf])
df_all.to_excel(r'C:\Users\no\Python Programs\abreu_network_robot\files\clientes_exchange_categorias.xlsx', index=False)